In [1]:
import dash
import pandas as pd
##from pandas_datareader import data as web
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import plotly.graph_objs as go
from dash import html
from dash import dcc
from dash import dash_table
##from dash import dbc
from dash.dash_table.Format import Group
#import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.linear_model import LinearRegression
##from fastai.tabular.all import *
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
from fastai.tabular.all import *
from plotly.subplots import make_subplots

In [2]:
def getStockData(value):
    if value == "AAPL":
##        return web.DataReader(value, data_source='yahoo', start='01-01-2019')
      return pd.read_csv("https://raw.githubusercontent.com/capwin3/UoS/main/AAPL.csv")
##df = web.DataReader(stock, data_source='yahoo', start='01-01-2019')
    elif value == "TSM":
      return pd.read_csv("https://raw.githubusercontent.com/capwin3/UoS/main/TSM.csv")
    elif value == "MSFT":
      return pd.read_csv("https://raw.githubusercontent.com/capwin3/UoS/main/MSFT.csv")
    elif value == "V":
      return pd.read_csv("https://raw.githubusercontent.com/capwin3/UoS/main/V.csv")
    elif value == "JNJ":
      return pd.read_csv("https://raw.githubusercontent.com/capwin3/UoS/main/JNJ.csv")

In [3]:
app = dash.Dash(__name__)


df = pd.DataFrame(columns=['Date','Open','High','Low','Close','Adj_Close','Volume','Ticker'])

app.layout = html.Div([
                      html.H1('Stock Prediction Analysis Dashboard', style={"textAlign": "center"}),
                      dcc.Tabs(id='tabs-example', value='tab-1', children=[
                            dcc.Tab(label='Prediction', value='tab-1', children=[
                                    dcc.Dropdown(
                                    id='demo-dropdown',
                                    options=[
                                    {'label': 'Apple', 'value': 'AAPL'},
                                    {'label': 'Taiwan Semiconductor Mfg', 'value': 'TSM'},
                                    {'label': 'Microsoft', 'value': 'MSFT'},
                                    {'label': 'Visa', 'value': 'V'},
                                    {'label': 'Johnson & Johnson', 'value': 'JNJ'}
                                    ],
                                    value='AAPL',
                                    multi=False
                                    ),
                                    html.Div(id='Message', children='Select a stock and press submit, it takes approximately 3 mins to finish.'),
                                      html.Button('Submit', id='submit-val', n_clicks=0),  
                                      dcc.Loading(
                                      id='loading1',
                                      children=html.Div([html.Div(id='chart1'),
                                                         html.Div(id='chart2'),
                                                         html.Div(id='chart3'),
                                                         html.Div(id='chart4'),
                                                         html.Div(id='chart5')
                                                        ])
                                        )
                            ]),
                      dcc.Tab(label='Raw Data', value='tab-2', children=[
                            html.Div(id='combo'),
                            html.Div(id='table')
#                            dash_table.DataTable(
#                            id='table',
#                                columns=[
#                                {'name': 'Date', 'id': '0'},
#                                {'name': 'Open', 'id': '1'},
#                                {'name': 'High', 'id': '2'},
#                                {'name': 'Low', 'id': '3'},
#                                {'name': 'Close', 'id': '4'},
#                                {'name': 'Adj_Close', 'id': '5'},
#                                {'name': 'Volume', 'id': '6'},
#                                {'name': 'Ticker', 'id': '7'}]
#                            {"name": i, "id": i} for i in sorted(df.columns)
#                            ],
#                            page_current=0,
#                            page_size=50,
#                            page_action='custom'
#                            )
                        ])
                    ])
])


In [4]:
@app.callback(
    Output('chart5', 'children'),
    Input('demo-dropdown', 'value'),
    Input('submit-val', 'n_clicks')
)
def Predict_LSTM(value, n_clicks):
    if n_clicks > 0:
        print(value)
        df_lstm=getStockData(value)
        df_train, df_valid=LSTM_Prediction(df_lstm)
        return generateLSTMPredict(df_train, df_valid)

In [5]:
def LSTM_Prediction(value):
    df=value
    df["Date"]=pd.to_datetime(df.Date,format="%m/%d/%Y")
    df.index=df['Date']

    df['Date'] = pd.to_datetime(df.Date,format='%Y-%m-%d')
    df.index = df['Date']
    
  #creating dataframe
    data = df.sort_index(ascending=True, axis=0)
    new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
    for i in range(0,len(data)):
        new_data['Date'][i] = data['Date'][i]
        new_data['Close'][i] = data['Close'][i]

    #setting index
    new_data.index = new_data.Date
    new_data.drop('Date', axis=1, inplace=True)

    #creating train and test sets
    dataset = new_data.values

    train = dataset[0:987,:]
    valid = dataset[987:,:]

    #converting dataset into x_train and y_train
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)

    x_train, y_train = [], []
    for i in range(60,len(train)):
        x_train.append(scaled_data[i-60:i,0])
        y_train.append(scaled_data[i,0])
    x_train, y_train = np.array(x_train), np.array(y_train)

    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
    model.add(LSTM(units=50))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

    #predicting 246 values, using past 60 from the train data
    inputs = new_data[len(new_data) - len(valid) - 60:].values
    inputs = inputs.reshape(-1,1)
    inputs  = scaler.transform(inputs)

    X_test = []
    for i in range(60,inputs.shape[0]):
        X_test.append(inputs[i-60:i,0])
    X_test = np.array(X_test)

    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
    closing_price = model.predict(X_test)
    closing_price = scaler.inverse_transform(closing_price)

    train = new_data[:987]
    valid = new_data[987:]
    valid['Predictions'] = closing_price
    
    x = 100
    
    return train, valid

In [6]:
def generateLSTMPredict(train,valid):
    print("generateLSTMPredict")
    return dcc.Graph(
        id='graph2',
                    figure={
                        "data":[
                            go.Scatter(
                                name='Train Data',
                                x=train.index,
                                y=train['Close'],
                                mode='lines'
                            ),
                            go.Scatter(
                                name='Predicted Close',
                                x=valid.index,
                                y=valid['Predictions'],
                                mode='lines'
                            ),
                            go.Scatter(
                                name='Actual Close',
                                x=valid.index,
                                y=valid['Close'],
                                mode='lines'
                            )
                        ],
                        "layout":go.Layout(
                            title='LSTM Prediction Plot',
                            xaxis={'title':'Date'},
                            yaxis={'title':'Price'}
                        )
                    }
                )                

In [7]:
@app.callback(
    Output('chart4', 'children'),
    Input('demo-dropdown', 'value'),
    Input('submit-val', 'n_clicks')
)
def Predict_ARIMA(value, n_clicks):
    if n_clicks > 0:
        print(value)
        df_arima=getStockData(value)
        df_train, df_valid, df_forecast=ARIMA_Prediction(df_arima)
        return generateARIMAPredict(df_train, df_valid, df_forecast)

In [8]:
def generateARIMAPredict(train,valid,forecast):
    print("generateARIMAPredict")
    return dcc.Graph(
        id='graph4',
                    figure={
                        "data":[
                            go.Scatter(
                                name='Train Data',
                                x=train.index,
                                y=train['Close'],
                                mode='lines'
                            ),
                            go.Scatter(
                                name='Predicted Close',
                                x=forecast.index,
                                y=forecast['Predictions'],
                                mode='lines'
                            ),
                            go.Scatter(
                                name='Actual Close',
                                x=valid.index,
                                y=valid['Close'],
                                mode='lines'
                            )
                        ],
                        "layout":go.Layout(
                            title='ARIMA Prediction Plot',
                            xaxis={'title':'Date'},
                            yaxis={'title':'Price'}
                        )
                    }
                )                

In [9]:
def ARIMA_Prediction(value):
    #achieve the df
    df = None
    
    df=value
    df["Date"]=pd.to_datetime(df.Date,format="%m/%d/%Y")
    df.index=df['Date']

    df['Date'] = pd.to_datetime(df.Date,format='%Y-%m-%d')
    df.index = df['Date']
    
    data = df.sort_index(ascending=True, axis=0)

    train = data[:987]
    valid = data[987:]

    training = train['Close']
    validation = valid['Close']

    model = auto_arima(training, start_p=1, start_q=1,max_p=3, max_q=3, m=12,start_P=0, seasonal=True,d=1, D=1, trace=True,error_action='ignore',suppress_warnings=True)
    model.fit(training)
    
    forecast = model.predict(n_periods=271)
    forecast = pd.DataFrame(forecast, columns=['Predictions'], index=valid.index)
 
    x = 75
##  df = df.loc[df1.index, :]  # works
      
    return train, valid, forecast

In [10]:
@app.callback(
    Output('chart3', 'children'),
    Input('demo-dropdown', 'value'),
    Input('submit-val', 'n_clicks')
)
def Predict_Linear(value, n_clicks):
    if n_clicks > 0:
        print(value)
        df_linear=getStockData(value)
        df_train, df_valid=Linear_Prediction(df_linear)
        return generateLinearPredict(df_train, df_valid)

In [11]:
def generateLinearPredict(train,valid):
    print("generateLinearPredict")
    return dcc.Graph(
        id='graph5',
                    figure={
                        "data":[
                            go.Scatter(
                                name='Train Data',
                                x=train.index,
                                y=train['Close'],
                                mode='lines'
                            ),
                            go.Scatter(
                                name='Predicted Close',
                                x=valid.index,
                                y=valid['Predictions'],
                                mode='lines'
                            ),
                            go.Scatter(
                                name='Actual Close',
                                x=valid.index,
                                y=valid['Close'],
                                mode='lines'
                            )
                        ],
                        "layout":go.Layout(
                            title='Linear Prediction Plot',
                            xaxis={'title':'Date'},
                            yaxis={'title':'Price'}
                        )
                    }
                )    

In [12]:
##from fastai.tabular import all
from fastai import *
def Linear_Prediction(value):
    df=value
    print("linear_predict")
    #setting index as date values
    df["Date"]=pd.to_datetime(df.Date,format="%m/%d/%Y")
    df.index=df['Date']

    df['Date'] = pd.to_datetime(df.Date,format='%Y-%m-%d')
    df.index = df['Date']
    
    #sorting
    data = df.sort_index(ascending=True, axis=0)

    #creating a separate dataset
    new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close']) ##, 'Date2'])

    for i in range(0,len(data)):
        new_data['Date'][i] = data['Date'][i]
        new_data['Close'][i] = data['Close'][i]
    
    #create features
    ##from fastai.structured import  add_datepart
##    from fastai.tabular import *
    add_datepart(new_data, 'Date')
    new_data.drop('Elapsed', axis=1, inplace=True)  #elapsed will be the time stamp
    
    new_data['mon_fri'] = 0
    for i in range(0,len(new_data)):
        if (new_data['Dayofweek'][i] == 0 or new_data['Dayofweek'][i] == 4):
        ##new_data['mon_fri'][i] = 1
            new_data.iloc[[i], 13] = 1
        else:
        ##new_data['mon_fri'][i] = 0
            new_data.iloc[[i], 13] = 0
        
    #split into train and validation
    train = new_data[:987]
    valid = new_data[987:]

    x_train = train.drop('Close', axis=1)
    y_train = train['Close']
    x_valid = valid.drop('Close', axis=1)
    y_valid = valid['Close']

    #implement linear regression
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(x_train,y_train)
    
    preds = model.predict(x_valid)
    
    pd.options.mode.chained_assignment = None  # default='warn'
    #plot
    valid['Predictions'] = 0
    valid['Predictions'] = preds

    valid.index = new_data[987:].index
    train.index = new_data[:987].index

    x = 50
    
    return train, valid

In [13]:
@app.callback(
    Output('chart1', 'children'),
    Input('demo-dropdown', 'value'),
    Input('submit-val', 'n_clicks')
)
def retStockData(value, n_clicks):
    print(value)
    if n_clicks > 0:
        df=getStockData(value)
        df["Date"]=pd.to_datetime(df.Date,format="%m/%d/%Y")
        df.index=df['Date']
        return generateCandleGraphs(df,value)

In [14]:
def generateCandleGraphs(df,value):
    print("generateCandleGraphs")
    # Calculate and define moving average of 30 periods
    avg_30 = df.Close.rolling(window=30, min_periods=1).mean()

    # Calculate and define moving average of 50 periods
    avg_50 = df.Close.rolling(window=50, min_periods=1).mean()
    trace1 = {
    'x': df.index,
    'open': df.Open,
    'close': df.Close,
    'high': df.High,
    'low': df.Low,
    'type': 'candlestick',
    'name': value,
    'showlegend': False
    }
    trace2 = {
    'x': df.index,
    'y': avg_30,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'blue'
            },
    'name': 'Moving Average of 30 periods'
    }
    trace3 = {
    'x': df.index,
    'y': avg_50,
    'type': 'scatter',
    'mode': 'lines',
    'line': {
        'width': 1,
        'color': 'red'
    },
    'name': 'Moving Average of 50 periods'
    }

    return dcc.Graph(
        id='graph6',
        figure = {
            "data": [trace1, trace2, trace3],
            "layout":go.Layout({
                'title': {
                'text': '"{}" Historial Closing Price'.format(value),
                'font': {
                'size': 15
            }
            }
        })
        })

In [15]:
@app.callback(
    Output('chart2', 'children'),
    Input('demo-dropdown', 'value'),
    Input('submit-val', 'n_clicks')
)
def Predict_Moving(value, n_clicks):
    if n_clicks > 0:
        print(value)
        df_moving=getStockData(value)
        df_train, df_valid=Moving_Average_Prediction(df_moving)
        return generateMovingPredict(df_train, df_valid)

In [16]:
def Moving_Average_Prediction(value):
    df_mov=value
    df_mov["Date"]=pd.to_datetime(df_mov.Date,format="%m/%d/%Y")
    df_mov.index=df_mov['Date']

    df_mov['Date'] = pd.to_datetime(df_mov.Date,format='%Y-%m-%d')
    df_mov.index = df_mov['Date']
    
    #creating dataframe with date and the target variable
    mov_data = df_mov.sort_index(ascending=True, axis=0)
    new_mov_data = pd.DataFrame(index=range(0,len(df_mov)),columns=['Date', 'Close'])

    for i in range(0,len(mov_data)):
         new_mov_data['Date'][i] = mov_data['Date'][i]
         new_mov_data['Close'][i] = mov_data['Close'][i]

# splitting into train and validation
    mov_train = new_mov_data[:987]
    mov_valid = new_mov_data[987:]
    
# In the next step, we will create predictions for the validation set and check the RMSE using the actual values.
# making predictions
    preds = []
    for i in range(0,mov_valid.shape[0]):
        a = mov_train['Close'][len(mov_train)-248+i:].sum() + sum(preds)
        b = a/248
        preds.append(b)

# checking the results (RMSE value)
##    rms=np.sqrt(np.mean(np.power((np.array(mov_valid['Close'])-preds),2)))
    pd.options.mode.chained_assignment = None  # default='warn'
    mov_valid['Predictions'] = 0
    mov_valid['Predictions'] = preds

    x = 25
    return mov_train, mov_valid

In [17]:
def generateMovingPredict(train,valid):
    print("generateMovingPredict")
    return dcc.Graph(
        id='graph3',
                    figure={
                        "data":[
                            go.Scatter(
                                name='Train Data',
                                x=train['Date'],
                                y=train['Close'],
                                mode='lines'
                            ),
                            go.Scatter(
                                name='Predicted Close',
                                x=valid['Date'],
                                y=valid['Predictions'],
                                mode='lines'
                            ),
                            go.Scatter(
                                name='Actual Close',
                                x=valid['Date'],
                                y=valid['Close'],
                                mode='lines'
                            )
                        ],
                        "layout":go.Layout(
                            title='Moving Average Prediction Plot',
                            xaxis={'title':'Date'},
                            yaxis={'title':'Price'}
                        )
                    }
                )      

In [18]:
@app.callback(
    Output('table', 'children'),
#    Output('table', 'page_size'),
#    Output('table', 'columns'),
    Input('demo-dropdown', 'value'),
    Input('submit-val', 'n_clicks'))
def update_table(value, n_clicks):
    if n_clicks > 0:
        df=getStockData(value)
        df['Ticker'] = value
        page_current=0,
        page_size=50,
#        df.iloc[
#            page_current*page_size:(page_current+ 1)*page_size
#            ].to_dict('records')
#           ].to_dict('rows')
        columns = [{"name": i, "id": i, } for i in df.columns]
        df_data = df.to_dict('rows')
#        fig.page_size=25
#        columns=[{"name": i, "id": i} for i in sorted(df.columns)],
        return dash_table.DataTable(data=df_data, columns=columns)

In [19]:
@app.callback(
    dash.dependencies.Output('combo', 'children'),
    dash.dependencies.Input('submit-val', 'n_clicks'))
def update_output(n_clicks):
    if n_clicks > 0:
#        fig = make_subplots(specs=[[{"secondary_y": True}]])
#        return dcc.Graph(figure=None)
#    else:
        df = pd.read_csv("https://raw.githubusercontent.com/capwin3/UoS/main/TSM.csv")

        fig = make_subplots(specs=[[{"secondary_y": True}]])#this a one cell subplot
        trace1 = go.Bar(x=df['Date'],
                y=df['Volume'],
                name='Volume',   
#                title='Closing and Volume'        
#                marker=dict(color= 'rgba(0,0,0)',      # 0.8)', 
#                            line= dict(width= 1))
               )
 

        trace2 = go.Scatter(x=df.Date, 
                    y=df.Close, 
#                    marker= dict(line= dict(width= 1), 
#                           size= 8), 
#                    line=dict(color= '#B0122C', width= 1.5),
                    name= 'Close')
        
#        trace3 = go.Layout(title='Historial Closing and Volume')
#The first trace is referenced to the default xaxis, yaxis (ie. xaxis='x1', yaxis='y1')
        fig.add_trace(trace1, secondary_y=False);

#The second trace is referenced to xaxis='x1'(i.e. 'x1' is common for the two traces) 
#and yaxis='y2' (the right side yaxis)

        fig.add_trace(trace2, secondary_y=True);
#        fig.add_trace(trace3);
        fig.update_layout(autosize= True, 
#                  width= 700, 
#                  height= 500, 
                  title="Historical Closing and Volume",
                  legend= dict(x=1.15, 
                               y=1),
                  hovermode='x');
        fig.update_xaxes(tickangle= -45, 
                 autorange=True,
#                 tickfont=dict(size= 10), 
#                 title= 'Tournaments', 
#                 type= 'category'
                );

        y1 = df["Volume"]
        max_y1 = y1.max()
        y2 = df["Close"]
        max_y2 = y2.max()

        fig.update_yaxes(range= [0, y1], #left yaxis
                 title= 'Volume', secondary_y=False);
        fig.update_yaxes(range= [0, y2],#right yaxis
                 showgrid= False, 
                 title= 'Close',
                 secondary_y=True);
        return dcc.Graph(figure=fig)

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port = 8080)

Dash is running on http://127.0.0.1:8080/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
AAPL
AAPL
AAPL
AAPL
AAPL
AAPL
generateCandleGraphs
Performing stepwise search to minimize aic
linear_predict


C:\PCH\anaconda3\lib\site-packages\pandas\core\frame.py:1549: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.



generateMovingPredict
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=inf, Time=1.83 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=3442.570, Time=0.24 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=3200.306, Time=0.48 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=0.83 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=3415.216, Time=0.17 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=3106.771, Time=0.67 sec
generateLinearPredict
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=inf, Time=2.29 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=inf, Time=0.90 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=3144.340, Time=0.40 sec
 ARIMA(2,1,0)(2,1,0)[12]             : AIC=3106.991, Time=1.11 sec
 ARIMA(1,1,1)(2,1,0)[12]             : AIC=3106.964, Time=1.50 sec
 ARIMA(0,1,1)(2,1,0)[12]             : AIC=3110.904, Time=0.73 sec
 ARIMA(2,1,1)(2,1,0)[12]             : AIC=3108.906, Time=2.43 sec
 ARIMA(1,1,0)(2,1,0)[12] intercept   : AIC=3108.763, Time=1.96 sec

Best model:  ARIMA(1,1,0)(2,1,0)[12] 